In [1]:
import numpy as np
import pandas as pd
from glob import glob
import sys
import os

import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [16, 9]
plt.rcParams['figure.dpi'] = 200


sys.path.append("..") 
from tp_utils import daily_returns
from tp_config import *
from tp_assets import *

## Helper functions

In [2]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

from datetime import datetime
now = datetime.today().strftime('%Y-%m-%d')

def weights_to_df(cleaned_weights):
    dfw = pd.DataFrame.from_dict([cleaned_weights]).transpose()
    dfw.columns = ['W']
    dfw = dfw[dfw['W'] > 0]
    return dfw

def final_sums(df, total):
    xx = round(df * total, -1)
    return xx[xx['W'] > 50]

In [16]:
assets = ['sp', 'msft', 'nflx', 'dis', 'ntdoy', 'goog', 'fb', 'amzn', 'rtn', 'gold', 'yndx', 'ba', 'baba', 'cpri', 'lmt', 'elisa.he']   
etoro_energy = ['A2A.MI', 'ATC.AS', 'AEE', 'AEP', 'ALB', 'ATO', 'AWK', 'BCPT.L', 'BRW.L', 'CCC.L', 'CIG', 'CMS', 'CNA.L', 'CNP', 'CVA',
               'D', 'DANSKE.CO', 'DC.L', 'DRI.DE', 'DTE', 'DUK',
                'ED', 'EDF.PA', 'EIX', 'ELE.MC', 'ELISA.HE', 'ENEL.MI', 'ENG.MC', 'ENGI.PA', 'ENS', 'EOAN.DE', 'ETR', 'EVA', 'EVRG', 'EXC',
                'FE', 'FGP.L', 'FORTUM.HE', 'GLO.L', 'GVC.L', 'HER.MI', 'IBE.MC', 'IG.MI', 'IGY.DE', 'ILD.PA', 'IRE.MI', 'LEO.DE', 'LNT',
               'MPC', 'MRC.L', 'NEE', 'NEP', 'NG.L', 'NI', 'NMC.L', 'NTGY.MC',  'OKE', 'ORA', 
                'PAM', 'PCG', 'PDL.L', 'PEG', 'PEGI', 'PPL', 'PSX']
outliers = ['lyft', 'tsla']

watch = ['ELISA.HE', 'CCC.L']

In [31]:
etoro_energy

['A2A.MI',
 'ATC.AS',
 'AEE',
 'AEP',
 'ALB',
 'ATO',
 'AWK',
 'BCPT.L',
 'BRW.L',
 'BWLPG.OL',
 'CCC.L',
 'CIG',
 'CMS',
 'CNA.L',
 'CNP',
 'CVA',
 'D',
 'DANSKE.CO',
 'DC.L',
 'DRI.DE',
 'DTE',
 'DUK',
 'ED',
 'EDF.PA',
 'EIX',
 'ELE.MC',
 'ELISA.HE',
 'ENEL.MI',
 'ENG.MC',
 'ENGI.PA',
 'ENS',
 'EOAN.DE',
 'ES',
 'ETR',
 'EVA',
 'EVRG',
 'EXC',
 'FE',
 'FGP.L',
 'FORTUM.HE',
 'GLO.L',
 'GVC.L',
 'HER.MI',
 'IBE.MC',
 'IG.MI',
 'IGY.DE',
 'ILD.PA',
 'IRE.MI',
 'LEO.DE',
 'LNT',
 'MPC',
 'MRC.L',
 'NEE',
 'NEP',
 'NG.L',
 'NI',
 'NMC.L',
 'NTGY.MC',
 'NVTK.IL',
 'OGZD.IL',
 'OKE',
 'ORA',
 'PAM',
 'PCG',
 'PDL.L',
 'PEG',
 'PEGI',
 'PPL',
 'PSX']

## Read from local data

In [17]:
data_path = DATA_PATH_SHARES

tickers = assets
res = []
for ticker in tickers:
    fn_full = os.path.join(data_path, ticker + '.csv')
    df = pd.read_csv(fn_full, index_col='Date', parse_dates=True)    
    df_close = df[['Adj Close']]
    df_close.columns = [ticker]
    res.append(df_close)

prices_df = pd.concat(res, axis = 1)
prices_df.tail()

,sp,msft,nflx,dis,ntdoy,goog,fb,amzn,rtn,gold,yndx,ba,baba,cpri,lmt,elisa.he
Date,,,,,,,,,,,,,,,,
2020-03-11,33.919998,153.630005,349.920013,105.510002,41.529999,1215.410034,170.240005,1820.859985,156.750000,18.790001,35.869999,189.080002,198.910004,18.83,356.929993,48.169998
2020-03-12,29.340000,139.059998,315.250000,91.809998,38.919998,1114.910034,154.470001,1676.609985,144.110001,16.879999,32.119999,154.839996,185.100006,14.34,311.380005,44.119999
2020-03-13,30.480000,158.830002,336.299988,102.519997,39.700001,1219.729980,170.279999,1785.000000,149.850006,15.670000,33.939999,170.199997,194.000000,13.93,328.589996,43.389999
2020-03-16,25.600000,135.419998,298.839996,95.010002,37.959999,1084.329956,146.009995,1689.150024,129.580002,16.330000,30.799999,129.610001,178.850006,9.66,287.940002,42.130001
2020-03-17,23.770000,146.570007,319.750000,93.529999,41.020000,1119.800049,149.419998,1807.839966,128.949997,17.709999,31.660000,124.139999,184.809998,8.15,318.829987,49.040001


In [29]:
# calculate expected returns and sample covariance amtrix
days = 512
df_train = prices_df.iloc[-days:]
avg_returns = expected_returns.mean_historical_return(df_train)
cov_mat = risk_models.sample_cov(df_train)
cov_mat;

In [30]:
# get weights maximizing the Sharpe ratio
ef = EfficientFrontier(avg_returns, cov_mat)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)
cleaned_weights = ef.clean_weights()
dfw = weights_to_df(cleaned_weights)
final_sums(dfw, 3000)

Expected annual return: 26.8%
Annual volatility: 19.2%
Sharpe Ratio: 1.29


,W
msft,1200.0
gold,780.0
elisa.he,1020.0


In [31]:
# get weights maximizing the Sharpe ratio
ef = EfficientFrontier(avg_returns, cov_mat)
weights = ef.min_volatility()
ef.portfolio_performance(verbose=True)
cleaned_weights = ef.clean_weights()
dfw = weights_to_df(cleaned_weights)
final_sums(dfw, 4000)

Expected annual return: 8.4%
Annual volatility: 15.9%
Sharpe Ratio: 0.40


,W
sp,410.0
dis,510.0
ntdoy,310.0
goog,130.0
rtn,400.0
gold,720.0
baba,230.0
elisa.he,1260.0


In [ ]:
from datetime import datetime
import backtrader as bt

class SmaCross(bt.SignalStrategy):
    def __init__(self):
        sma1, sma2 = bt.ind.SMA(period=10), bt.ind.SMA(period=30)
        crossover = bt.ind.CrossOver(sma1, sma2)
        self.signal_add(bt.SIGNAL_LONG, crossover)

cerebro = bt.Cerebro()
cerebro.addstrategy(SmaCross)

data0 = bt.feeds.YahooFinanceData(dataname='AAPL', fromdate=datetime(2011, 1, 1),
                                  todate=datetime(2012, 12, 31))
cerebro.adddata(data0)

cerebro.run()
cerebro.plot()